In [1]:
import pandas as pd
from Bio import SeqIO
import numpy as np
import os
import random

Ideas of the program would be:

- want to read in reference_dataframe.csv

- taxonomic level e.g. genus=Candida -> species level comparision

- path to location of data. Root of analysis/.....

With arg parse

ref_df_fn = args.ref_df_fn  
data_root = args.data_root  
tax_rank = args.tax_rank #use with flag e.g. -r  
name = args.name #name of the rank to select from, use flag e.g. -n  

In [2]:
ref_df_fn = '../../analysis/Stats/reference_dataframe.csv'
ref_df_fn = os.path.abspath(ref_df_fn)

In [3]:
try:
    os.path.exists(ref_df_fn)
except:
    print('Cannot find %s' % ref_df_fn)

In [ ]:
#to be able to handle anything from command lien
tax_rank = 'family'
name = 'Saccharomycetaceae'

In [ ]:
n_reads = 15000

In [ ]:
name = name.lower()
tax_rank = tax_rank.lower()

In [4]:
ref_df = pd.read_csv(ref_df_fn, index_col=None)

In [ ]:
ref_df[ref_df["# reads after length filtering"] < n_reads].shape[0]

In [ ]:
try:
    if ref_df[ref_df["# reads after length filtering"] \
              < n_reads].shape[0] >0 :
        print("These species need more reads.")
        print(ref_df[ref_df["# reads after length filtering"] \
              < n_reads])
        #exit()
except:
    print('Check %s to have the wanted column names' % ref_df_fn)

In [ ]:
#not neccessary pull from command line
# name = "Trichocomaceae"
data_root = '../../'

In [ ]:
indecies = ref_df[ref_df[tax_rank] == name].index

In [ ]:
indecies

In [ ]:
SeqIO_dicts = {}
for index in indecies:
    fasta_path = os.path.join(data_root, ref_df.loc[index, 'path to length filtering'])
    print(fasta_path)
    try:
        SeqIO_dicts[index] = SeqIO.to_dict(SeqIO.parse(fasta_path, 'fasta'))
    except:
        print('Check location of fasta files.')

In [ ]:
def max_seq_len(SeqIO_dict):
    """
    Fucntion takes a SeqIO_dict and returns the lenghts of the
    longest sequence.
    """
    total_lens = []
    for key in SeqIO_dict.keys():
        total_lens.append(len(SeqIO_dict[key].seq))
    return max(total_lens)

In [ ]:
def numberfi(SeqIO_dict, seq_len, nsubsample):
    """
    Take SeqIO_dict and return SeqIO_dict were bases have been replaced
    with numbers
    ACGT- to 01234
    Take the seq_len each sequence should have.
    """
    num_dict = {}
    
    keys = list(SeqIO_dict.keys())
    randkeys = random.sample(keys, k= nsubsample)
    
    
    for key in randkeys:
        seq = str(SeqIO_dict[key].seq).replace("A",'0 ')\
        .replace("C",'1 ').replace("G",'2 ').replace("T",'3 ')
        seq_new = seq + '4 '*(seq_len -int(len(seq)/2))
        num_dict[key] = list(map(int, seq_new.split(' ')[:-1]))
    return num_dict
    

In [ ]:
total_lens = []
for key, value in SeqIO_dicts.items():
    total_lens.append(max_seq_len(value))
print("This is the maximum sequence length of all sampled sequences.")
print(total_lens)

numSeqIO_dicts = {}
max_len = max(total_lens)
for key, value in SeqIO_dicts.items():
    numSeqIO_dicts[key] = numberfi(value, max_len, s_reads)

seq_list = []
for index in indecies:
    seq_list.append(np.array(list(numSeqIO_dicts[index].values())))
seq_comb = np.concatenate(seq_list, axis = 0)

In [ ]:
num_class = len(numSeqIO_dicts.keys())
ids_comb = np.zeros( (n_reads*num_class,num_class) )
for i in range(0, num_class):
    ids_comb[i*n_reads:(i+1)*n_reads,i] = 1


In [ ]:
num_class = 5
ids_comb = np.zeros( (n_reads*num_class,num_class) )

In [ ]:
for i in range(0, num_class):
    ids_comb[i*n_reads:(i+1)*n_reads,i] = 1

In [ ]:
ids_comb

In [5]:
ref_df.head()

,Unnamed: 0,species,genus,family,order,class,phylum,kingdom,# raw reads,# reads after homology filtering,# reads after length filtering,# reads for use,path to raw reads,path to homology filtering,path to length filtering,path for use
0,20171103_FAH15473/barcode01,striiformis-tritici,puccinia,pucciniaceae,pucciniales,pucciniomycetes,basidiomycota,fungi,272465,122080,113337,113337.0,analysis/Concatenated/20171103_FAH15473/barcod...,analysis/Python_Processing/20171103_FAH15473/b...,analysis/Length_Filtered/20171103_FAH15473/bar...,analysis/Length_Filtered/20171103_FAH15473/bar...
1,20171103_FAH15473/barcode02,tritici,zymoseptoria,mycosphaerellaceae,capnodiales,dothideomycetes,ascomycota,fungi,413127,143363,133089,133089.0,analysis/Concatenated/20171103_FAH15473/barcod...,analysis/Python_Processing/20171103_FAH15473/b...,analysis/Length_Filtered/20171103_FAH15473/bar...,analysis/Length_Filtered/20171103_FAH15473/bar...
2,20171103_FAH15473/barcode03,tritici-repentis,pyrenophora,pleosporaceae,pleosporales,dothideomycetes,ascomycota,fungi,260896,97584,90015,90015.0,analysis/Concatenated/20171103_FAH15473/barcod...,analysis/Python_Processing/20171103_FAH15473/b...,analysis/Length_Filtered/20171103_FAH15473/bar...,analysis/Length_Filtered/20171103_FAH15473/bar...
3,20171103_FAH15473/barcode04,oxysporum,fusarium,nectriaceae,hypocreales,sordariomycetes,ascomycota,fungi,382450,131411,123742,123742.0,analysis/Concatenated/20171103_FAH15473/barcod...,analysis/Python_Processing/20171103_FAH15473/b...,analysis/Length_Filtered/20171103_FAH15473/bar...,analysis/Length_Filtered/20171103_FAH15473/bar...
4,20171103_FAH15473/barcode05,brumale,tuber,tuberaceae,pezizales,pezizomycetes,ascomycota,fungi,275035,80614,74232,74232.0,analysis/Concatenated/20171103_FAH15473/barcod...,analysis/Python_Processing/20171103_FAH15473/b...,analysis/Length_Filtered/20171103_FAH15473/bar...,analysis/Length_Filtered/20171103_FAH15473/bar...


In [6]:
print(type(ref_df.loc[0,'# reads after length filtering']))

<class 'numpy.int64'>


In [7]:
print(type(ref_df.loc[0,'# reads for use']))

<class 'numpy.float64'>


In [ ]:
ref_df[tax_rank] == 'Candida'

In [ ]:
ref_df.head()

In [ ]:
location = (ref_df.columns.get_loc(tax_rank)-1)

In [ ]:
classes = ref_df.iloc[indecies,location].unique()

In [ ]:
count_dict = {}
for item in classes:
    count_dict[item] = sum(ref_df.iloc[:,location] == item)

In [ ]:
classes

In [ ]:
count_dict

In [ ]:
for item in count_dict.keys():
    print(n_reads/count_dict[item])

In [ ]:
# define n_reads

min_vals = []
for key, value in count_dict.items():
    if value == min(count_dict.values()):
        min_vals.append(ref_df[ref_df.iloc[:,location] == item]['# reads after length filtering'].min())
min(min_vals)

In [ ]:
max_reads = 0
for key, value in count_dict.items():
    if value == max(count_dict.values()):
        max_reads = value*n_reads

In [ ]:
max_reads

In [ ]:
min_vals = [44999]

In [ ]:
# is max_reads < min_reads?
# if yes, use max_reads to subsample as base value

In [ ]:
if max_reads <= min(min_vals):
    for key, value in count_dict.items():
        s_reads = int(max_reads/value)
        print(key, s_reads)
else:
    for key, value in count_dict.items():
        s_reads = int(min(min_vals)/value)
        print(key, s_reads)

In [ ]:
location = (ref_df.columns.get_loc(tax_rank)-1)
col_name = ref_df.columns[location]

classes = ref_df.iloc[indecies,location].unique()

count_dict = {}
for item in classes:
    count_dict[item] = sum(ref_df.iloc[:,location] == item)

min_vals = []
for key, value in count_dict.items():
    if value == min(count_dict.values()):
        min_vals.append(ref_df[ref_df.iloc[:,location] == item]['# reads after length filtering'].min())
minimum_value = min(min_vals)

max_reads = 0
for key, value in count_dict.items():
    if value == max(count_dict.values()):
        max_reads = value*n_reads

if max_reads <= minimum_value:
    minimum_value = max_reads
for key, n_class in count_dict.items():
    s_reads = int(minimum_value/n_class)
    if ref_df[ref_df.loc[:,col_name]==key]['# reads after length filtering'].min() < s_reads:
        minimum_value = ref_df[ref_df.loc[:,col_name]==key]['# reads after length filtering'].min()/n_class
        s_reads = int(minimum_value/n_class)
    print(key, s_reads)
    
# parse s_reads into numberfy

In [ ]:
tax_ranks = ref_df.columns[2:-6]
print(tax_ranks)

In [ ]:
for tax_rank in tax_ranks:
    print(tax_rank + "\n\n")
    for name in ref_df[tax_rank].unique():
        total_class_reads = []
        
        print("\n\n"+name)
        indecies = ref_df[ref_df[tax_rank] == name].index
        location = (ref_df.columns.get_loc(tax_rank)-1)
        col_name = ref_df.columns[location]

        classes = ref_df.iloc[indecies,location].unique()
        print(classes)

        count_dict = {}
        for class_ in classes:
            count_dict[class_] = sum(ref_df.iloc[indecies,location] == class_)
        print(count_dict)

        min_vals = []
        for class_, n_class in count_dict.items():
            if n_class == min(count_dict.values()):
                min_vals.append(ref_df[ref_df.iloc[:,location] == class_]['# reads after length filtering'].min())
        if min(min_vals) % 2 == 0:
            minimum_value = min(min_vals)
        else:
            minimum_value = min(min_vals)-1
        print("minimum_value =", minimum_value)
        
        if len(count_dict) > 1:
            max_reads = 0
            for key, value in count_dict.items():
                if value == max(count_dict.values()):
                    max_reads = value*n_reads

            if max_reads <= minimum_value:
                minimum_value = max_reads
                
            for key, n_class in count_dict.items():
                s_reads = int(minimum_value/n_class)
                if ref_df[ref_df.loc[:,col_name]==key]['# reads after length filtering'].min() < s_reads:
                    minimum_value = ref_df[ref_df.loc[:,col_name]==key]['# reads after length filtering'].min()/n_class
                    s_reads = int(minimum_value/n_class)
                print(key, s_reads)
                total_class_reads.append(s_reads*n_class)
            print(total_class_reads)
        elif len(count_dict) == 1:
            s_reads = n_reads
            print(s_reads)
#             print("no comparison for the rank")
#             exit()

        # parse s_reads into numberfy